# Notebook 1: Introduction to Pandas and the Titanic Data Set
***

If you haven't at least skimmed the Numpy and Pandas tutorial, **STOP** and go do that now. 

In this notebook you'll apply some basic Pandas tools to explore the ubiquitous **Titanic** dataset. 

First, as always, we'll load Numpy and Pandas using their common aliases, np and pd. 

In [1]:
import numpy as np 
import pandas as pd

The data is stored in a .csv file (a format that lists data separated by commas) called clean_titanic_data.csv.  We'll import the data into Pandas using the read_csv( ) function.  

In [2]:
# Path to the data - select the path that works for you 
file_path = 'clean_titanic_data.csv'

# Load the data into a DataFrame 
df = pd.read_csv(file_path)

Take a look at the first few rows of the DataFrame using the head( ) method. 

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


From this you should see that each row in the DataFrame refers to a particular passenger on the Titanic.  The columns of the DataFrame give you specific information about each passenger.  The **PassengerId** is simply a unique identifier given to each passenger in the data set.  The rest of the attributes are more meaningful: 

- **Survived**: Indicates whether the passenger survived the sinking
- **Pclass**: Indicates the socio-economic status of the passenger (lower number means higher class)
- **Name**: The passenger's name 
- **Sex**: The passenger's sex 
- **Age**: The passenger's age
- **SibSp**: The number of siblings / spouses the passenger was traveling with 
- **Parch**: The number of children / parents the passenger was traveling with 
- **Ticket**: The passenger's ticket number 
- **Fare**: How much the passenger paid for their ticket 
- **Embarked**: The passenger's port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

### Exercise 1
***
Determine how many people survived the disaster

In [36]:
people_surv =df.index[df["Survived"]==1]
num_surv=people_surv.size
print("{} people survived the disaster".format(num_surv))


surv= df.groupby(["Sex", "Survived"], as_index= False)["Survived"].count()
surv

290 people survived the disaster


,Sex,Survived
0,female,64
1,female,197
2,male,360
3,male,93


### Exercise 2  
***
Determine how many men and how many women survived the disaster 

In [38]:
male_survived = df.index[df["Sex"]=="male"] & people_surv 
female_survived =df.index[df["Sex"]=="female"] & people_surv
print("{} men survived the disaster".format(male_survived.size))
print("{} women survived the disaster".format(female_survived.size))

93 men survived the disaster
197 women survived the disaster


C:\Users\julia\AppData\Local\Temp/ipykernel_16328/1009779465.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  male_survived = df.index[df["Sex"]=="male"] & people_surv
C:\Users\julia\AppData\Local\Temp/ipykernel_16328/1009779465.py:2: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  female_survived =df.index[df["Sex"]=="female"] & people_surv


### Exercise 3 
***
Determine how many children ages 12 and under survived the disaster. 

In [70]:
#since boolean we can use sum
#also could do group by
children_survived = df.loc[df['Age']<=12, 'Survived'].sum()
print("{} children survived the disaster".format(children_survived))

total_children = df.loc[df['Age']<=12].index.size
#could equivalenly do total_children = len(df.loc[df['Age']<=12])
print("{} children survived the disaster".format(total_children))

40 children survived the disaster
69 children survived the disaster


**Question to ponder:**  Reflect on your answers to Exercises 1 and 3.  Do you think being a child makes you more or less likely to survive the sinking of the Titanic?

**Answer:**  Exercise 1 showed that 290/714 = 40.6% of passengers overall survived, and Exercise 3 showed that 40/69 = 58.0% of children survived.  Since this is greater than the overall survival proportion, we would hypothesize that being a child makes someone **more likely** to survive the sinking of the Titanic.

Later in this course, we will learn how to formally test your hypothesis.

However, the exitement can start... $\color{red}{\text{now}}$!

### Exercise 4 
***
The **SibSp** and **Parch** attributes tell us the number of siblings/spouses and parents/children each passenger had on board.  Create a new column in the DataFrame called **Family** that indicates how many siblings/spouses/parents/children a passenger was traveling with. Then report how many people survived that were traveling with 3 or more family members. 

In [87]:
df["Family"] = df["SibSp"]+df["Parch"]
family_survived =df.loc[df["Family"]>=3 ,"Survived"].sum() ##people with family is 78
#family and survived is 31
##should be 31

print("{} people traveling with 3 or more family members survived the disaster".format(family_survived))

31 people traveling with 3 or more family members survived the disaster


### Exercise 5 
***
In this exercise you will write some code to predict whether a person survived the disaster based on their information.  Obviously, you'll want to ignore the **Survived** attribute for this to avoid cheating. You'll store your predictions ($1$ if you predict survived, $0$ if you predict died) in a column of the DataFrame called **Prediction**.  You can then use the following function to see how accurate your prediction was. 

In [88]:
def score_prediction(df):
    '''
    Function to score predictions.  
    Takes entire DataFrame as sole argument. 
    '''
    acc = (df["Survived"]==df["Prediction"]).sum() / len(df)
    print("Your accuracy is {0:.1f}%".format(100*acc))

There are several ways that we can accomplish this.  The first method we'll highlight is one that loops over every row in the DataFrame, makes a decision based on that row's attributes, and then sets the relevant prediction in the **Prediction** column.  As an example, we'll use a very naive heuristic that predicts that all males survive and all females die. 

In [10]:
for passenger_index, passenger in df.iterrows():
    df.loc[passenger_index, "Prediction"] = 1 if df.loc[passenger_index, "Sex"] == 'male' else 0

We can check that our code actually did something using the head( ) function and observing that we do in fact have a column called **Prediction** populated with $1$'s and $0$'s. You can see that the $1$'s in the **Prediction** column do in-fact line up with "male" in the **Sex** column. 

In [11]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Family,Prediction
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0,1.0
5,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S,0,1.0
6,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,S,4,1.0
7,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,S,2,0.0
8,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,C,1,0.0
9,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,S,2,0.0


Now, let's see how we did by passing our DataFrame into the score_prediction( ) function. 

In [12]:
score_prediction(df)

Your accuracy is 22.0%


And here we see that our naive prediction netted us a 22% prediction accuracy (which isn't very good, but you're going to make it better). 

OK, so looping over the data is one option, but in Python, unfortunately, it's not a very good option.  Python is an interpreted language, which means that loops are slow.  We didn't really notice it here, because our DataFrame only has around 700 rows in it, but on data sets with hundreds of thousands or millions of entries, loops can grind your day to a complete halt.  

It's better to use vectorized methods like Pandas apply( ) function combined with Python lambda functions.  One way to accomplish the same results as above is as follows: 

In [13]:
df["Prediction"] = df["Sex"].apply(lambda s: 1 if s=="male" else 0)
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Family,Prediction
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0,1
5,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,S,0,1
6,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,S,4,1
7,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,S,2,0
8,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,C,1,0
9,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,S,2,0


You can check that this produces the same result as the loop-based method. We can compare the speeds of the apply( ) method and the loop-based method using the Jupyter magic %timeit command.  


In [14]:
# Loop-based method 
print("Timing loop-based method: ")
%timeit for passenger_index, passenger in df.iterrows(): df.loc[passenger_index, "Prediction"] = 1 if df.loc[passenger_index, "Sex"] == 'male' else 0
    
# Apply-based method 
print("Timing apply-based method: ")
%timeit df["Prediction"] = df["Sex"].apply(lambda s: 1 if s=="male" else 0)

Timing loop-based method: 
435 ms ± 48.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Timing apply-based method: 
1.08 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Note that here ms is milliseconds and $\mu\textrm{s}$ is **micro**seconds.  You can hopefully see that the apply method is **tremendously** faster than the loop-based method. 

OK, let's do one more slightly more complicated example so that we can see how to use the apply( ) function with multiple inputs.  Suppose say that you want to predict that a person survived if they are male **AND** they were traveling alone (probably not a good heuristic but just go with it).  To do this we need values from both the **Sex** column and the **Family** column.  Here is the code

In [50]:
df["Prediction"] = df.apply(lambda row: 1 if row["Sex"]=="male" and row["Family"]==0 else 0, axis=1)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Family,Prediction
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0,1


Here the apply( ) function is applied to the entire DataFrame and the object passed to the lambda function is an entire row of the DataFrame.  We can then carve off the elements from the columns we're interested in and do our thing.  Let's see how we did! 

In [51]:
score_prediction(df)

Your accuracy is 31.1%


**W00T!** Minor improvement! OK, so you're job is to explore the data and see if you can cook up a custom prediction heuristic that does better than 31.1%!  If you get an accuracy you're proud of, make a post about it on Piazza along with a description of what you did! 